# LLM agents for literature review
Get information about scientific papers and their citations, and check if they are relevant to the PSC disease

## Import libaries and prepare the API credential

In [1]:
pip install pyautogen==0.3.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.



> Feature 1 Snowballing

In [3]:


from autogen import ConversableAgent, register_function, GroupChatManager, GroupChat
from autogen.coding import LocalCommandLineCodeExecutor

with open('openai.credential', 'r') as file:
    key = file.read()

MODEL = 'gpt-4o'

# Decide if there is human interaction or not
DEBUG_MODE = False


/opt/conda/lib/python3.11/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [4]:
from neo4j import GraphDatabase

gdb = GraphDatabase.driver('neo4j://neo4j-lumen', auth = ("neo4j", "llm-agents"))
gdb.verify_connectivity()

def query_neo4j(query:str)->str:

    try:
        records, summary, keys = gdb.execute_query(query)
        return repr(records)
    except Exception as e:
        return repr(e) 
    

graph_tool_executor = ConversableAgent(
    "graph_tool_executor",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False,
    human_input_mode= "ALWAYS" if DEBUG_MODE else "NEVER",
    is_termination_msg=lambda msg: (msg["content"]) and ("terminate" in msg["content"].lower())
)

graph_tool_driver = ConversableAgent(
    "graph_tool_driver",
    system_message = "You have connection to a Neo4j Graph Database, and you can answer questions by querying the database. To do so, you need to generate Cypher queries and use a registered tool to execute the query and retrive the knowledge you need from the graph."
    "The graph has the following schema:"
    "An ASSEST has properties name, layer, ip and description."
    "An ASSET can DestributesTo or ConnectTo another ASSET."
    "A DATASOURCE has properties name, type, format, bucket, and endpoint, and is DataSourceOf an ASSET"
    "A STATICDATA has properties name, type, file_name, add_date, format, and is DataOf an ASSET"
    "be careful about the functions of the relationships, i.e., DATASOURCE-[:DataSourceOf]->ASSET, and STATICDATA-[:DataOf]->[ASSET]",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False,
    human_input_mode= "ALWAYS" if DEBUG_MODE else "NEVER"
)

register_function(
    query_neo4j,
    caller = graph_tool_driver,
    executor = graph_tool_executor,
    description = "Query or modify the neo4j graph database. The input is a cypher query, and the output is a list of records returned from the query."
)

nested_chats = [
    {
        "recipient": graph_tool_driver,
        "max_turns": 4,
        "summary_method": "last_msg"
    }
]

graph_tool_executor.register_nested_chats(
    nested_chats, 
    trigger = lambda sender: sender not in [graph_tool_driver]
)

### Human interface agent

> Feature 3: a human chat interface

In [5]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False,
    human_input_mode= "ALWAYS"
)

## Feature 5: Chatbot for reading the paper graph

In [6]:
def start_chat():
    chat_result = graph_tool_executor.initiate_chat(
        human_proxy,
        message="Tell me what you want to know about the data graph?"
    )

In [7]:

start_chat()


graph_tool_executor (to human_proxy):

Tell me what you want to know about the data graph?

--------------------------------------------------------------------------------
human_proxy (to graph_tool_executor):

what is the endpoint to access the datasource of asset CSMS?

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
graph_tool_executor (to graph_tool_driver):

what is the endpoint to access the datasource of asset CSMS?

--------------------------------------------------------------------------------
graph_tool_driver (to graph_tool_executor):

***** Suggested tool call (call_c7OnxWwtA2w2ddKfpKhvT1d2): query_neo4j *****
Arguments: 
{"query":"MATCH (a:ASSET {name: 'CSMS'})<-[:DataSourceOf]-(d:DATASOURCE) RETURN d.endpoint"}
******************************************